# Python Fundamentals for Seismology — Notebook 7
## NumPy for Seismology (arrays, windows, metrics, missing data, masked arrays)

*Python Fundamentals for Seismology*

Run cells in order. Edit parameters and re-run to explore.*


## Learning objectives
- Understand NumPy arrays and how they differ from Python lists
- Use indexing/slicing to work with waveform windows
- Apply vectorised operations for speed and clarity
- Compute common signal metrics (mean, RMS, peak-to-peak)
- Handle missing data using NaNs and (later) masked arrays
- Understand masked arrays (`np.ma`) for gappy data and QC


## 0. Setup
We'll use NumPy and Matplotlib. Everything here runs without ObsPy.
Later you can plug in real traces (e.g., `tr.data`) as NumPy arrays.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=4, suppress=True)


## 1. Lists vs NumPy arrays
A Python list is a general-purpose container.
A NumPy array is a **homogeneous**, **typed** block of memory designed for numerical work.
That makes arrays faster and enables vectorised operations.


In [ ]:
py_list = [1, 2, 3, 4]
arr = np.array(py_list, dtype=float)

print("list:", py_list, type(py_list))
print("array:", arr, type(arr), arr.dtype, arr.shape)


In [ ]:
# Vectorised arithmetic (elementwise)
x = np.array([1, 2, 3], dtype=float)
print("x + 10:", x + 10)
print("x * 2 :", x * 2)

# With lists, this does something else:
y = [1, 2, 3]
print("list + list:", y + y)     # concatenation, not numeric addition


## 2. Creating time vectors and synthetic seismograms
We'll generate a simple synthetic waveform: a sinusoid + noise + a transient.
This stands in for real `Trace.data` arrays.


In [ ]:
fs = 100.0  # sample rate (Hz)
dt = 1.0 / fs
t = np.arange(0, 60, dt)  # 60 seconds

rng = np.random.default_rng(0)
noise = 0.2 * rng.standard_normal(t.size)

# A low-frequency component + noise
x = 0.8*np.sin(2*np.pi*1.0*t) + noise

# Add a transient at ~30 s (e.g., an event onset)
onset = int(30 * fs)
x[onset:onset+50] += 2.0*np.hanning(50)

print("n_samples:", x.size)
print("duration_s:", x.size/fs)


In [ ]:
plt.figure()
plt.plot(t, x)
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (arbitrary)")
plt.title("Synthetic waveform")
plt.show()


## 3. Indexing and slicing (waveform windows)
Slicing is central to waveform work:
- select a time window
- select a pre-event noise window
- select a signal window

Convert times to sample indices: `i = int(time * fs)`.


In [ ]:
# Pick a 10-second window around the onset (25–35 s)
t0, t1 = 25.0, 35.0
i0, i1 = int(t0*fs), int(t1*fs)

x_win = x[i0:i1]
t_win = t[i0:i1]

print("window samples:", x_win.size)
print("window duration:", x_win.size/fs)


In [ ]:
plt.figure()
plt.plot(t_win, x_win)
plt.axvline(30, linestyle="--")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Windowed waveform (25–35 s)")
plt.show()


## 4. Common signal metrics (vectorised)
A few very common metrics in seismology workflows:
- mean
- standard deviation
- RMS
- peak-to-peak


In [ ]:
def rms(a):
    a = np.asarray(a)
    return np.sqrt(np.mean(a*a))

mean = np.mean(x_win)
std = np.std(x_win)
r = rms(x_win)
p2p = np.ptp(x_win)  # max - min

print("mean:", mean)
print("std :", std)
print("rms :", r)
print("p2p :", p2p)


## 5. Vectorisation vs loops (why NumPy matters)
If you find yourself writing a loop that does arithmetic on each sample, try NumPy operations instead.


In [ ]:
# Example: rectify a waveform (absolute value)
abs_vec = np.abs(x_win)

# Same idea with a loop (works, but slower and noisier code)
abs_loop = np.empty_like(x_win)
for i in range(x_win.size):
    abs_loop[i] = abs(x_win[i])

print("max difference:", np.max(np.abs(abs_vec - abs_loop)))


## 6. Boolean masks for QC and picking
Boolean arrays let you select samples meeting a condition.
This is a core pattern for quality control.


In [ ]:
# Example QC: flag large outliers
threshold = 2.5
mask_outlier = np.abs(x) > threshold

print("outlier count:", np.sum(mask_outlier))
print("first few indices:", np.where(mask_outlier)[0][:10])


In [ ]:
plt.figure()
plt.plot(t, x, label="x")
plt.plot(t[mask_outlier], x[mask_outlier], ".", label="flagged")
plt.xlabel("Time (s)")
plt.title("Boolean mask flags outliers")
plt.legend()
plt.show()


## 7. Detrending (simple) with NumPy
For many quick analyses, removing a linear trend can help.
ObsPy has `tr.detrend('linear')`; here’s the NumPy idea.


In [ ]:
# Fit a line to the whole record: x ~ a*t + b
a, b = np.polyfit(t, x, 1)
trend = a*t + b
x_detrended = x - trend

print("trend slope:", a)

plt.figure()
plt.plot(t, x, label="original")
plt.plot(t, trend, label="fitted trend")
plt.xlabel("Time (s)")
plt.title("Linear trend fit")
plt.legend()
plt.show()

plt.figure()
plt.plot(t, x_detrended)
plt.xlabel("Time (s)")
plt.title("Detrended waveform")
plt.show()


## 8. Simple smoothing via convolution (moving average)
A moving average is a crude low-pass filter and a good way to introduce convolution.


In [ ]:
M = int(0.5 * fs)  # 0.5-second moving average
h = np.ones(M)/M

x_smooth = np.convolve(x_detrended, h, mode="same")

plt.figure()
plt.plot(t, x_detrended, label="detrended")
plt.plot(t, x_smooth, label="smoothed")
plt.xlim(24, 36)
plt.xlabel("Time (s)")
plt.title("Moving average smoothing")
plt.legend()
plt.show()


## 9. NaNs as missing data (quick-and-dirty)
Often you’ll represent gaps as `np.nan`. But beware:
- many NumPy functions will return `nan` if any `nan` is present
- use `np.nanmean`, `np.nanstd`, etc.


In [ ]:
x_nan = x.copy()

# Insert a synthetic gap: 10–12 s
g0, g1 = int(10*fs), int(12*fs)
x_nan[g0:g1] = np.nan

print("mean (plain):", np.mean(x_nan))
print("mean (nanmean):", np.nanmean(x_nan))


In [ ]:
plt.figure()
plt.plot(t, x_nan)
plt.xlabel("Time (s)")
plt.title("Waveform with NaN gap")
plt.show()


## 10. Masked arrays (`np.ma`) for gappy data and QC
Masked arrays let you keep the original numeric values *and* a mask saying which samples are invalid.

Why masked arrays can be nicer than NaNs:
- you can preserve original values but mark them invalid
- many operations automatically ignore masked values
- you can mask based on QC rules (clip level, dropouts, timing gaps)

Concept:
- `data`: numeric array
- `mask`: boolean array (`True` means *masked out / invalid*)


In [ ]:
# Create a mask: True where values are invalid
mask = np.isnan(x_nan)

x_ma = np.ma.array(x_nan, mask=mask)

print("masked count:", x_ma.mask.sum())
print("mean ignoring masked:", x_ma.mean())  # np.ma methods ignore masked values
print("std ignoring masked :", x_ma.std())


In [ ]:
# You can also mask by QC rules:
# Example: mask outliers as well as NaNs
mask_qc = np.isnan(x_nan) | (np.abs(x_nan) > 2.5)
x_ma_qc = np.ma.array(x_nan, mask=mask_qc)

print("masked count (QC):", x_ma_qc.mask.sum())
print("mean (QC):", x_ma_qc.mean())


In [ ]:
# Masked arrays for plotting:
# Matplotlib will typically not draw masked samples (creates gaps).
plt.figure()
plt.plot(t, x_ma_qc)
plt.xlabel("Time (s)")
plt.title("Masked array (NaNs + QC outliers masked)")
plt.show()


### Converting back and forth
- To fill masked values with a number (e.g., 0): `x_ma.filled(0.0)`
- To get the underlying data: `x_ma.data`
- To get the mask: `x_ma.mask`


In [ ]:
filled = x_ma_qc.filled(0.0)
print("filled sample around gap:", filled[g0-3:g0+3])

print("data dtype:", x_ma_qc.data.dtype)
print("mask dtype:", x_ma_qc.mask.dtype)


### Important nuance for seismology workflows
- If you plan to run **FFT / filtering**, you usually need a fully numeric array (no mask). You must decide how to fill gaps.
- For **summary statistics**, **QC**, and **feature extraction**, masked arrays are often a clean solution.
- ObsPy `Trace` objects typically store raw data as a regular NumPy array; you can create a masked view when needed.


## Exercises
1. Compute RMS of a *noise window* (e.g., 20–25 s) and a *signal window* (30–32 s). Compute an SNR = RMS(signal)/RMS(noise).
2. Mask clipped samples: choose a clip level (e.g., abs(x) > 2.0) and compute mean/std ignoring them.
3. Create a function `window_by_time(x, fs, t0, t1)` returning `x[i0:i1]`.


In [ ]:
# 1) SNR from RMS windows
def rms(a):
    a = np.asarray(a)
    return np.sqrt(np.mean(a*a))

# TODO: pick windows and compute SNR
snr = None
print("SNR:", snr)

# 2) Mask clipped samples and compute stats
# TODO

# 3) window_by_time
def window_by_time(x, fs, t0, t1):
    # TODO
    pass


### Solutions (peek after trying)


In [ ]:
def rms(a):
    a = np.asarray(a)
    return np.sqrt(np.mean(a*a))

def window_by_time(x, fs, t0, t1):
    i0, i1 = int(t0*fs), int(t1*fs)
    return x[i0:i1]

noise = window_by_time(x, fs, 20, 25)
signal = window_by_time(x, fs, 30, 32)
snr = rms(signal) / rms(noise)
print("SNR:", snr)

clip_level = 2.0
mask_clip = np.abs(x) > clip_level
x_ma_clip = np.ma.array(x, mask=mask_clip)
print("masked:", x_ma_clip.mask.sum())
print("mean ignoring clipped:", x_ma_clip.mean())
print("std ignoring clipped :", x_ma_clip.std())
